In [1]:
import pandas as pd
import gzip
import ast
from sklearn.metrics.pairwise import cosine_similarity

In [54]:
archivo = "../02 - Desanidado de datos y limpieza/Juegos_ETL.csv.gz"
with gzip.open(archivo, "rt", encoding='utf-8') as file:
    df = pd.read_csv(file)

In [55]:
df.head()

,Unnamed: 0,Id_juego,Nombre_juego,Genero,Año_Lanzamiento,Formato,Pago
0,0,761140,Lost Summoner Kitty,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",2018,['Single-player'],1
1,1,643980,Ironbound,"['Free to Play', 'Indie', 'RPG', 'Strategy']",2018,"['Single-player', 'Multi-player', 'Online Mult...",0
2,2,670290,Real Pool 3D - Poolians,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",2017,"['Single-player', 'Multi-player', 'Online Mult...",0
3,3,767400,弹炸人2222,"['Action', 'Adventure', 'Casual']",2017,['Single-player'],1
4,5,772540,Battle Royale Trainer,"['Action', 'Adventure', 'Simulation']",2018,"['Single-player', 'Steam Achievements']",1


In [69]:
Juegos = df[["Id_juego", "Nombre_juego", "Genero",]]
Juegos.head()

,Id_juego,Nombre_juego,Genero
0,761140,Lost Summoner Kitty,"['Action', 'Casual', 'Indie', 'Simulation', 'S..."
1,643980,Ironbound,"['Free to Play', 'Indie', 'RPG', 'Strategy']"
2,670290,Real Pool 3D - Poolians,"['Casual', 'Free to Play', 'Indie', 'Simulatio..."
3,767400,弹炸人2222,"['Action', 'Adventure', 'Casual']"
4,772540,Battle Royale Trainer,"['Action', 'Adventure', 'Simulation']"


In [45]:
Juegos.shape

(27334, 3)

In [70]:
Juegos_muestra = Juegos.sample(frac=0.10, random_state=22)
Juegos_muestra

,Id_juego,Nombre_juego,Genero
22956,350330,Call of the Ninja!,"['Casual', 'Indie']"
19453,466000,Maze: Subject 360 Collector's Edition,"['Adventure', 'Casual']"
3841,376770,Big Thinkers 1st Grade,['Casual']
17256,546360,Driver Booster 4 Upgrade to Pro (Lifetime),['Utilities']
22366,355970,Home is Where One Starts...,"['Adventure', 'Indie']"
...,...,...,...
16232,582580,Itineris,"['Adventure', 'Casual', 'Indie']"
5029,421670,CUPID - A free to play Visual Novel,"['Free to Play', 'Indie', 'RPG', 'Simulation']"
24559,290401,Light Aurora Pack,['RPG']
26222,97379,Magic 2013 “Crosswinds” Foil Conversion,['Strategy']


In [61]:
# Convertir las cadenas que representan listas a listas reales
Juegos_muestra['Genero'] = Juegos_muestra['Genero'].apply(ast.literal_eval)

In [62]:
# Aplicar one-hot encoding a las columnas 'Genero' y 'Formato'
df_expandido_genero = Juegos_muestra['Genero'].apply(lambda x: pd.Series(1, x)).fillna(0)

# Eliminar las columnas originales 'Genero' y 'Formato'
Juegos_muestra = Juegos_muestra.drop(['Id_juego', 'Genero'], axis=1)

# Combina el DataFrame expandido con el original
Juego_resultado = pd.concat([Juegos_muestra, df_expandido_genero], axis=1)

Juego_resultado

,Nombre_juego,Casual,Indie,Adventure,Utilities,Strategy,RPG,Free to Play,Simulation,Action,...,Racing,Sports,Design &amp; Illustration,Education,Software Training,Web Publishing,Animation &amp; Modeling,Video Production,Photo Editing,Audio Production
22956,Call of the Ninja!,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19453,Maze: Subject 360 Collector's Edition,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3841,Big Thinkers 1st Grade,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17256,Driver Booster 4 Upgrade to Pro (Lifetime),0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22366,Home is Where One Starts...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16232,Itineris,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5029,CUPID - A free to play Visual Novel,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24559,Light Aurora Pack,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26222,Magic 2013 “Crosswinds” Foil Conversion,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
Juego_resultado = Juego_resultado.set_index('Nombre_juego')
Juego_resultado

,Casual,Indie,Adventure,Utilities,Strategy,RPG,Free to Play,Simulation,Action,Massively Multiplayer,...,Racing,Sports,Design &amp; Illustration,Education,Software Training,Web Publishing,Animation &amp; Modeling,Video Production,Photo Editing,Audio Production
Nombre_juego,,,,,,,,,,,,,,,,,,,,,
Call of the Ninja!,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Maze: Subject 360 Collector's Edition,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Big Thinkers 1st Grade,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Driver Booster 4 Upgrade to Pro (Lifetime),0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Home is Where One Starts...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Itineris,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CUPID - A free to play Visual Novel,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Light Aurora Pack,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
def recomendacion_juego(id_juego):

    Juego_resultado
    Similitud_coseno = cosine_similarity(Juego_resultado.values)
    
    df_sc = pd.DataFrame(Similitud_coseno, index = Juego_resultado.index, columns = Juego_resultado.index)

    df_sc.columns = df["Id_juego"]

    # Obtener la columna correspondiente al juego dado
    columna_juego = df_sc[id_juego]
    
    # Obtener los juegos con los mejores puntajes (menores que 1)
    Juegos = columna_juego[columna_juego < 1.0].sort_values(ascending=False).head(5).index.tolist()

    return Juegos

In [52]:
id_juego = 643980
resultados = recomendacion_juego(id_juego)

print(f"Juegos recomendados similares a ID n°'{id_juego}': {resultados}")

c:\Users\atenc\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


Juegos recomendados similares a ID n°'643980': ['Dungeon Defenders II - Standard Edition', 'Labyrinth', 'Dungeon Defenders II - Bag Booster Pack', 'Dungeon Defenders II - Gemtacular Pack', 'Emporea: Realms of War and Magic']


In [64]:
Similitud_coseno = cosine_similarity(Juego_resultado.values)
Similitud_coseno

array([[1.        , 0.5       , 0.70710678, ..., 0.        , 0.        ,
        0.63245553],
       [0.5       , 1.        , 0.70710678, ..., 0.        , 0.        ,
        0.63245553],
       [0.70710678, 0.70710678, 1.        , ..., 0.        , 0.        ,
        0.4472136 ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.4472136 ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.4472136 ],
       [0.63245553, 0.63245553, 0.4472136 , ..., 0.4472136 , 0.4472136 ,
        1.        ]])

In [65]:
df_sc = pd.DataFrame(Similitud_coseno, index = Juego_resultado.index, columns = Juego_resultado.index)
df_sc

Nombre_juego,Call of the Ninja!,Maze: Subject 360 Collector's Edition,Big Thinkers 1st Grade,Driver Booster 4 Upgrade to Pro (Lifetime),Home is Where One Starts...,Geneforge 3,Sentinels of the Multiverse - Mini-Pack 2,Insanity Clicker,Sally's Law,Rocksmith® 2014 – Stone Temple Pilots - “Big Empty”,...,Voyage: Journey to the Moon,Depth - Corsair Pack,Crusader Kings II: Mongol Faces,HardCube Soundtrack,Blues and Bullets - Episode 2,Itineris,CUPID - A free to play Visual Novel,Light Aurora Pack,Magic 2013 “Crosswinds” Foil Conversion,Cora (Character for Occult preRaise)
Nombre_juego,,,,,,,,,,,,,,,,,,,,,
Call of the Ninja!,1.000000,0.500000,0.707107,0.0,0.500000,0.408248,0.500000,0.534522,1.000000,0.500000,...,0.500000,0.500000,0.000000,0.816497,0.000000,0.816497,0.353553,0.000000,0.000000,0.632456
Maze: Subject 360 Collector's Edition,0.500000,1.000000,0.707107,0.0,0.500000,0.000000,0.000000,0.534522,0.500000,0.500000,...,1.000000,0.000000,0.000000,0.816497,0.707107,0.816497,0.000000,0.000000,0.000000,0.632456
Big Thinkers 1st Grade,0.707107,0.707107,1.000000,0.0,0.000000,0.000000,0.000000,0.377964,0.707107,0.707107,...,0.707107,0.000000,0.000000,0.577350,0.000000,0.577350,0.000000,0.000000,0.000000,0.447214
Driver Booster 4 Upgrade to Pro (Lifetime),0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Home is Where One Starts...,0.500000,0.500000,0.000000,0.0,1.000000,0.408248,0.500000,0.534522,0.500000,0.000000,...,0.500000,0.500000,0.000000,0.816497,0.707107,0.816497,0.353553,0.000000,0.000000,0.632456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Itineris,0.816497,0.816497,0.577350,0.0,0.816497,0.333333,0.408248,0.654654,0.816497,0.408248,...,0.816497,0.408248,0.000000,1.000000,0.577350,1.000000,0.288675,0.000000,0.000000,0.774597
CUPID - A free to play Visual Novel,0.353553,0.000000,0.000000,0.0,0.353553,0.577350,0.353553,0.755929,0.353553,0.353553,...,0.000000,0.353553,0.000000,0.288675,0.000000,0.288675,1.000000,0.500000,0.000000,0.447214
Light Aurora Pack,0.000000,0.000000,0.000000,0.0,0.000000,0.577350,0.000000,0.377964,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,1.000000,0.000000,0.447214


In [71]:
df_sc.columns = Juegos_muestra["Id_juego"]
df_sc

Id_juego,350330,466000,376770,546360,355970,200990,410480,393530,512230,390417,...,302180,414180,203773,500780,384380,582580,421670,290401,97379,741281
Nombre_juego,,,,,,,,,,,,,,,,,,,,,
Call of the Ninja!,1.000000,0.500000,0.707107,0.0,0.500000,0.408248,0.500000,0.534522,1.000000,0.500000,...,0.500000,0.500000,0.000000,0.816497,0.000000,0.816497,0.353553,0.000000,0.000000,0.632456
Maze: Subject 360 Collector's Edition,0.500000,1.000000,0.707107,0.0,0.500000,0.000000,0.000000,0.534522,0.500000,0.500000,...,1.000000,0.000000,0.000000,0.816497,0.707107,0.816497,0.000000,0.000000,0.000000,0.632456
Big Thinkers 1st Grade,0.707107,0.707107,1.000000,0.0,0.000000,0.000000,0.000000,0.377964,0.707107,0.707107,...,0.707107,0.000000,0.000000,0.577350,0.000000,0.577350,0.000000,0.000000,0.000000,0.447214
Driver Booster 4 Upgrade to Pro (Lifetime),0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Home is Where One Starts...,0.500000,0.500000,0.000000,0.0,1.000000,0.408248,0.500000,0.534522,0.500000,0.000000,...,0.500000,0.500000,0.000000,0.816497,0.707107,0.816497,0.353553,0.000000,0.000000,0.632456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Itineris,0.816497,0.816497,0.577350,0.0,0.816497,0.333333,0.408248,0.654654,0.816497,0.408248,...,0.816497,0.408248,0.000000,1.000000,0.577350,1.000000,0.288675,0.000000,0.000000,0.774597
CUPID - A free to play Visual Novel,0.353553,0.000000,0.000000,0.0,0.353553,0.577350,0.353553,0.755929,0.353553,0.353553,...,0.000000,0.353553,0.000000,0.288675,0.000000,0.288675,1.000000,0.500000,0.000000,0.447214
Light Aurora Pack,0.000000,0.000000,0.000000,0.0,0.000000,0.577350,0.000000,0.377964,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,1.000000,0.000000,0.447214


In [76]:
def recomendacion_juego2(id_juego):

    # Obtener la columna correspondiente al juego dado
    columna_juego = df_sc[id_juego]
    
    # Obtener los juegos con los mejores puntajes (menores que 1)
    Juegos = columna_juego[columna_juego < 1.0].sort_values(ascending=False).head(5).index.tolist()

    return Juegos

In [78]:
id_juego = 350330
resultados = recomendacion_juego2(id_juego)

print(f"Juegos recomendados similares a ID n°'{id_juego}': {resultados}")

Juegos recomendados similares a ID n°'350330': ['Call of the Ninja!', 'Underground Keeper', 'Cube Link', 'Pony World 3', 'Zasa - An AI Story']


In [72]:
df_sc.to_csv("../03 - Dataframe para funciones/recomendacion_juego.csv", index = True)